# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
from google_key import google_key as api
import matplotlib.pyplot as plt
from citipy import citipy
import scipy.stats as sp
import pandas as pd
import numpy as np
import requests
import gmaps
import os

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../Weather/output_data/cities.csv")
df.loc[:, ~df.columns.str.contains('^Unnamed')]

,City,Country,Date,Latitude,Longitude,Max Temp(f),Humidity,Wind Speed,Cloudiness
0,Nikolskoye,RU,1586068209,59.70,30.79,37.00,59,17.90,0
1,Arraial do Cabo,BR,1586068184,-22.97,-42.03,74.97,70,9.17,8
2,Jamestown,US,1586068198,42.10,-79.24,46.99,81,6.93,90
3,Nālūt,LY,1586068264,30.33,10.85,49.87,53,9.17,18
4,Busselton,AU,1586068181,-33.65,115.33,72.00,34,3.53,100
...,...,...,...,...,...,...,...,...,...
548,Palanga,LT,1586068384,55.92,21.07,41.00,93,3.36,4
549,Padang,ID,1586068210,-0.95,100.35,80.74,80,6.71,100
550,Ponta Delgada,PT,1586068384,37.73,-25.67,59.00,82,2.24,20
551,Damara,CF,1586068385,4.96,18.70,73.40,100,5.30,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure Gmaps
gmaps.configure(api_key=api)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = df[["Latitude", "Longitude"]].astype(float)

In [5]:
# Initialize (Heat)map
fig = gmaps.figure()

gmap_layer = gmaps.heatmap_layer(
    locations, 
    weights=df["Humidity"],
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(gmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#     - A max temperature lower than 80 degrees but higher than 70.
#     - Wind speed less than 10 mph.
#     - Zero cloudiness.
#     - Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by 
#       your API requests to a reasonable number.

In [7]:
vacation_spot = df[
    (df['Max Temp(f)'] > 70) & 
    (df['Max Temp(f)'] < 80) &
    (df['Wind Speed'] < 10.00) &
    (df['Cloudiness'] > 0) &
    (df['Cloudiness'] < 5)
]
vacation_spot

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temp(f),Humidity,Wind Speed,Cloudiness
54,54,Coahuayana Viejo,MX,1586068276,18.73,-103.68,71.64,54,4.14,2
99,99,Kapaa,US,1586068205,22.08,-159.32,75.20,88,4.03,1
257,257,Luangwa,ZM,1586068320,-15.62,30.42,74.98,63,3.53,4
394,394,Coolum Beach,AU,1586068348,-26.53,153.10,78.01,73,1.99,1
462,462,Loíza,PR,1586068365,18.43,-65.88,73.99,94,1.05,1
464,464,Kohlu,PK,1586068365,29.90,69.25,74.95,17,4.25,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store into hotel_df
hotel_df = pd.DataFrame(data=vacation_spot)

In [9]:
# Add a "Hotel Name" column into the DataFrame
hotel_df.loc[:,'Hotel Name'] = np.nan

In [10]:
# Set parameters to search for hotels within 5000 meters
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

In [11]:
# Hit the Google Places API for each city's coordinates

hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Latitude"].astype(str) +"," + hotel_df.loc[:,"Longitude"].astype(str)

hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": api
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5km of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]

,City,Country,Date,Latitude,Longitude,Max Temp(f),Humidity,Wind Speed,Cloudiness,Hotel Name,Coordinates
54,Coahuayana Viejo,MX,1586068276,18.73,-103.68,71.64,54,4.14,2,Hotel Los Arcos,"18.73,-103.68"
99,Kapaa,US,1586068205,22.08,-159.32,75.20,88,4.03,1,Sheraton Kauai Resort at Coconut Beach,"22.08,-159.32"
257,Luangwa,ZM,1586068320,-15.62,30.42,74.98,63,3.53,4,"Justfeeli Lodge, Luangwa","-15.62,30.42"
394,Coolum Beach,AU,1586068348,-26.53,153.10,78.01,73,1.99,1,The Beach Retreat Coolum,"-26.53,153.1"
462,Loíza,PR,1586068365,18.43,-65.88,73.99,94,1.05,1,Malibu Beachfront,"18.43,-65.88"
464,Kohlu,PK,1586068365,29.90,69.25,74.95,17,4.25,1,Killi Haji Sahib Dad Marri Kohlu,"29.9,69.25"


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map

new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations, hotel_info)
new_fig.add_layer(gmap_layer)
new_fig.add_layer(markers)
new_fig

Figure(layout=FigureLayout(height='420px'))